In [1]:
import pandas as pd
from itertools import product
from tqdm.notebook import tqdm

import sys, os

sys.path.append(os.path.abspath('..'))
%load_ext autoreload
%autoreload 2
from modules.config import *
from modules.svm import *

In [2]:
all_possible_metas = [
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100],  'gamma': [-1],                          'degree': [-1],         'max_iter': [1000000]},
    {'kernel': ['rbf'],    'C': [0.1, 1, 10, 100],  'gamma': [0.1, 0.01, 0.001, 0.0001],    'degree': [-1],         'max_iter': [1000000]},
    {'kernel': ['poly'],   'C': [1, 10, 100, 1000], 'gamma': [-1],                          'degree': [2, 3, 4, 5], 'max_iter': [1000000]}
]

In [3]:
execute_stage(
    SVM_FIRST_STAGE_DEMAND_RESULTS_PATH,
    SVM_FIRST_STAGE_DEMAND_RESULTS_PATH,
    SVM_SECOND_STAGE_DEMAND_RESULTS_PATH,
    TUNE_H3_RESOLUTION,
    TUNE_TIME_INTERVAL_LENGTH,
    all_possible_metas,
    get_availabe_models_metas_first_stage,
    do_evaluate_model=False,
    silent=False,
)

0it [00:00, ?it/s]

In [4]:
results = pd.read_parquet(SVM_FIRST_STAGE_DEMAND_RESULTS_PATH)
results.sort_values(by=['mean_test_score'], ascending=False).head(5)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,param_max_iter,params,...,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score,n_iter,h3_res,time_interval_length,param_gamma,param_degree
0,0,8290,54.808515,0.629360,11.699974,0.465198,1.0,rbf,1000000,"{'C': 1.0, 'degree': None, 'gamma': 0.01, 'ker...",...,-1.240730,-1.419944,-1.493995,-1.319479,0.116704,434001,8,6,0.010,NaN
3,0,8290,61.887783,2.143711,13.117010,0.575450,10.0,rbf,1000000,"{'C': 10.0, 'degree': None, 'gamma': 0.01, 'ke...",...,-0.968602,-1.101078,-1.162574,-1.023994,0.091734,0,8,6,0.010,NaN
7,0,8290,61.977852,1.651096,8.008226,0.643548,100.0,rbf,1000000,"{'C': 100.0, 'degree': None, 'gamma': 0.001, '...",...,-1.251774,-1.431241,-1.494112,-1.327835,0.114344,0,8,6,0.001,NaN
17,0,8290,72.496359,2.332309,11.504219,0.713604,10.0,poly,1000000,"{'C': 10.0, 'degree': 2.0, 'gamma': None, 'ker...",...,-1.278043,-1.453530,-1.513180,-1.352580,0.110961,0,8,6,NaN,2.0
12,2,74610,9881.552610,513.575073,284.903122,5.123617,10.0,rbf,1000000,"{'C': 10.0, 'degree': None, 'gamma': 0.01, 'ke...",...,-1.367001,-1.403957,-1.402731,-1.397316,0.020037,0,8,6,0.010,NaN


In [5]:
results.sort_values(by='mean_test_score', ascending=False).groupby('param_kernel').mean_test_score.first()

param_kernel
linear   -1.662285
poly     -1.508812
rbf      -1.465577
Name: mean_test_score, dtype: float64

In [6]:
results.groupby('param_kernel').mean_fit_time.sum() / 60

param_kernel
linear    214.667233
poly      476.428271
rbf       949.590948
Name: mean_fit_time, dtype: float64

In [7]:
resolutions = list(product(PREDICTIVE_H3_RESOLUTIONS, CALC_TIME_INTERVAL_LENGTHS))
resolutions = resolutions + (ADDITIONAL_PREDICTIVE_RESOLUTIONS)
resolutions

[(7, 1), (7, 2), (7, 6), (7, 24), (8, 1), (8, 2), (8, 6), (8, 24), (9, 24)]

In [8]:
for h3_res, time_interval_length in tqdm(resolutions):
    execute_stage(
        SVM_SECOND_STAGE_DEMAND_RESULTS_PATH,
        SVM_FIRST_STAGE_DEMAND_RESULTS_PATH,
        SVM_FIRST_STAGE_DEMAND_RESULTS_PATH,
        h3_res,
        time_interval_length,
        all_possible_metas,
        get_availabe_models_metas_second_stage,
        do_evaluate_model=True,
        silent=True,
    )

  0%|          | 0/9 [00:00<?, ?it/s]

In [10]:
results = pd.read_parquet(SVM_SECOND_STAGE_DEMAND_RESULTS_PATH)
results.sort_values(by=['mean_test_score'], ascending=False)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,param_max_iter,...,mean_train_score,std_train_score,n_iter,h3_res,time_interval_length,test_mse,test_rmse,test_mae,test_non_zero_mape,test_zero_accuracy
6,0,1000,0.138981,0.023714,0.027800,0.002046,100.0,0.01,rbf,1000000,...,-0.011098,0.002192,2917,8,1,0.357892,0.598241,0.377427,0.318250,0.992131
10,0,1000,1.080025,0.080829,0.156397,0.045222,100.0,0.01,rbf,1000000,...,-0.013832,0.002471,2943,9,24,0.419469,0.647664,0.370820,0.317879,0.986657
7,0,1000,0.230601,0.046164,0.036395,0.008022,100.0,0.01,rbf,1000000,...,-0.012916,0.002334,5331,8,2,0.847163,0.920415,0.595610,0.466315,0.948988
8,0,1000,0.159820,0.031024,0.038597,0.002787,100.0,0.01,rbf,1000000,...,-0.293319,0.103430,7439,8,6,2.730777,1.652506,1.062367,0.690098,0.919408
2,0,1000,0.101653,0.002476,0.096095,0.091909,100.0,0.01,rbf,1000000,...,-0.369825,0.169252,9413,7,1,6.988222,2.643525,1.724084,0.757214,0.909324
9,0,1000,0.151198,0.023317,0.032998,0.002003,100.0,0.01,rbf,1000000,...,-1.147964,0.169143,7833,8,24,7.933485,2.816644,1.740889,0.788139,0.924982
0,0,1000,0.107074,0.002481,0.056548,0.069059,100.0,0.01,rbf,1000000,...,-0.708350,0.359225,9905,7,1,6.842036,2.615729,1.725693,0.757921,0.917131
1,0,1000,0.122981,0.003790,0.022158,0.001010,100.0,0.01,rbf,1000000,...,-4.778296,0.851035,12444,7,2,17.420339,4.173768,2.667091,0.854532,0.913255
3,0,1000,0.133484,0.002250,0.023280,0.000470,100.0,0.01,rbf,1000000,...,-6.118797,0.997080,13139,7,2,17.485346,4.181548,2.655201,0.850723,0.915189
4,0,1000,0.112803,0.001949,0.022809,0.000391,100.0,0.01,rbf,1000000,...,-51.211343,7.912550,12355,7,6,78.378238,8.853148,4.718980,0.863182,0.944876
